ESTIMATING CORRELATION MATRICES

AUTHOR: [@sparshsah](https://github.com/sparshsah)

What is the best (lowest-MSE, and, ideally, also unbiased) way to estimate cross-correlations between return series?

Notes:
* You could be doing directional (TS, meaning you are timing individual sectors or assets) strategies, in which case you will tend to be either all-long or all-short assets within the same sector, so that it would be conservative to overestimate correlations (otherwise, if you underestimate, you won't realize how exposed you are to a macro shock). Or you could be doing relative-value (XS, meaning pairs trading) strategies, in which case it would be conservative to underestimate correlations (otherwise, if you overestimate, you'll think you're very tightly-hedged and not realize how exposed you are to a spreads-blowout). But these are risk-management considerations, which isn't our focus here.
* Keep in mind also that this is the foundation of quant strategies, but _not_ the most controversial part. This is the "risk" side of the equation, like the denominator in the Sharpe ratio. It's important to be thoughtful, but ultimately most answers will essentially agree here (largely because people have spent a long time figuring out the right way to do this, and risk---whether volatility or correlation---tends to be more stable over time despite being subject to violent spikes). The controversial side is the "reward" side, the numerator, which is in a way trying to capture the purely orthogonal component of returns, the predictable part as opposed to the unpredictable part (remember that for Normal data, the sample mean is independent of the sample standard deviation ([ref](http://www2.stat.duke.edu/courses/Fall18/sta611.01/Lecture/lec12_mean_var_indep.pdf))). You might hear that "signal-to-noise", the "Sharpe", is very low in finance: Another way of saying this is that the magnitude of the daily return's mean is very small relative to the magnitude of its uncertainty. I can say with great confidence that tomorrow's SPX return won't exceed 5\% in magnitude, i.e. that it will fall between [-5\%, +5\%]. But I am essentially split 50-50 on whether it will be positive or negative. In fact, I'm split 50-50 despite _knowing_ that it's likely to be positive! We know that SPX returns are negatively-skewed, and that despite that, the annualized ER is around +10\%, so the median return _has_ to be positive. But +10\% annualized ER corresponds to a daily ER literally less than +5bp. Let's say generously that the daily median return is twice that (in reality, it's only about 1.5x that), and put it at +10bp. That means my interval above shifts from [-5\%, +5\%] to (drumroll please)... [-4.9\%, +5.1\%]. So let's call that an average day, then we can say that a good day looks like [-4.8\%, +5.2\%] while a bad day looks like [-5.1\%, +4.9\%] (where I'm defining "good" as "Two Sigma Compass is long" and "bad" as "Two Sigma Compass is short"). Those two days look almost identical to each other, and so it's almost impossible even ex-post to distinguish them. Say yesterday's return came in at +1\%. Can you based on that say with any confidence that yesterday was, ex-ante, a good day? Or conversely can you reject, with any confidence, an argument that it was ex-ante a bad day that just got lucky?

# GLOBAL MACRO

The setting here is essentially that you have a medium-sized cross-section of only moderately-correlated assets, say O(100) assets where the magnitude of correlation is no more than 0.90 (so that $R^2 \approx 80\%$).

The classic example here is a global-macro strategy: You could have 5 developed country-level stock indices, 5 intra-EU country-level stock indices, 10 emerging country-level stock indices, 10 each of DEV and EMG government bonds and currencies, and 50 commodities (Energies, Base Metals, Precious Metals, Softs/Ags, Livestock).

Your goal here is going to be to regularize estimates that come deceptively high by random chance.

## Eigencleanup ([ref](https://arxiv.org/abs/1610.08104))

Leverage Random-Matrix Theory and in particular Marchenko-Pastur. You basically do an eigendecomposition, zero out the "noisy-looking" eigenvalues, then reassemble. Very similar to the process of "making" a matrix nonnegative-definite by zero'ing out negative eigenvalues. The latter arises when e.g. You have a candidate covariance matrix with each entry calculated in a slightly-different way or over a slightly-different subsample. E.g. For some assets you might want to use a full-sample estimate, but for others use a EWM estimate. Or, you might have a full timeseries for 99 assets, but only a couple years of recent history for the last asset -- You can add that last asset in using correlations estimated over whatever subsample you have data for, but you don't want to semi-blind yourself by throwing away the earlier data for literally every other asset just because of this last one. This scenario can give rise to a self-inconsistent covariance matrix that needs to be cleaned up.

## Shrinkage

For volatilities, you can just take a 65-day-COM EWM estimate and call it a day, perhaps applying some flooring. But for correlations, you're going to want to do something like multiply the off-diagonal elements by e.g. 0.90.

## Squashing

Map each day's return to its sign (`-1`, `0`, or `+1`). This will dampen the influence of single-day shocks, e.g. out-of-the-blue SPX had a +10\% day six weeks ago, and now every other asset looks zero-correlated to it because none of them had commensurate random spikes that day.

# STOCK SELECTION

Even though I'm calling it "stock selection", the setting here is simply that you have a large-sized cross-section of assets, say O(1,000), some of which are very strongly positively correlated with each other. In particular, the cross-section divides naturally into peer groups (e.g. regions or industries) of O(10) assets, and within each groups, cross-asset correlations can be as high as 0.95-0.99.

Classic examples:
* A global yield-curve strategy. You still have ~20 global govie issuers, some of which (e.g. US and Canada) are meaningfully-correlated with each other, say 0.90. But now, you go much more granular: Within each curve, you trade a combination of short-term-interest-rate and bond futures contracts to fill out the tenors. Every futures contract can expire on the next IMM date, but the underlying assets are, say, 3m, 6m, 9m, 1Y, 2Y, 5Y, 10Y, 20Y, 30Y, and Ultra-Long-Dated yields. From 20 yield curves you now have 200 securities, and clearly e.g. US 9m yields are going to tend to move in lockstep with US 6m yields, creating very tight return correlation for the corresponding STIR contracts.
* The namesake of this category, global stock-selection. Imagine filtering to US microcap tech stocks, and then choosing the ten that look "growth"-iest (as in Value-vs-Growth). You should expect very high correlations. In fact, I'd half-argue that the only reason you shouldn't expect _perfect_ correlations is that there are going to be so few people trading these stocks that how quickly their prices react to news will depend on whether the people trading them were busy eating breakfast when it broke.

## Factor-loading models

I haven't ever actually constructed such a risk model, but Barra is the industry standard.